In [1]:
from transformers import pipeline

summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


In [6]:
# Read test text
with open("test.txt", "r") as f:
    text = f.readlines()
    # Delete lines with :
    text = [line for line in text if ':' not in line]
    # Delete empty lines
    text = [line for line in text if line.strip()]
    # Join lines
    text = " ".join(text)
    # Limit number of words to 1024
    text = ' '.join(text.split()[:800])


In [7]:
print(text)

like yeah so i've flown nine times on the affectionately known as the vomit comet it's the parabolic flight and essentially it does what you'd want a plane never to do it pitches really steeply upwards at 45 degrees that's a picture you yeah yeah that's test rate that's super early in my phd some of just the passive tiles before we even put electronics in we were just testing the magnet polarity and the essentially is it an energy favorable structure to self-assemble on its own so we tweaked a lot of things between are we looking at a couple of them yeah you're looking at a bunch of them there's also almost 32 of them yeah they're clumping they're clumping yeah can you comment on what's the difference between microgravity and and zero gravity yes so there is it's important difference there is no zero gravity there's no nothing there's in the universe there is no such thing as zero gravity so newton's law of gravity tells us that there's always gravity attraction between any two objects

In [1]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = T5ForConditionalGeneration.from_pretrained("Michau/t5-base-en-generate-headline")
tokenizer = T5Tokenizer.from_pretrained("Michau/t5-base-en-generate-headline")
model = model.to(device)

In [8]:

article = '''
Very early yesterday morning, the United States President Donald Trump reported he and his wife First Lady Melania Trump tested positive for COVID-19. Officials said the Trumps' 14-year-old son Barron tested negative as did First Family and Senior Advisors Jared Kushner and Ivanka Trump.
Trump took to social media, posting at 12:54 am local time (0454 UTC) on Twitter, "Tonight, [Melania] and I tested positive for COVID-19. We will begin our quarantine and recovery process immediately. We will get through this TOGETHER!" Yesterday afternoon Marine One landed on the White House's South Lawn flying Trump to Walter Reed National Military Medical Center (WRNMMC) in Bethesda, Maryland.
Reports said both were showing "mild symptoms". Senior administration officials were tested as people were informed of the positive test. Senior advisor Hope Hicks had tested positive on Thursday.
Presidential physician Sean Conley issued a statement saying Trump has been given zinc, vitamin D, Pepcid and a daily Aspirin. Conley also gave a single dose of the experimental polyclonal antibodies drug from Regeneron Pharmaceuticals.
According to official statements, Trump, now operating from the WRNMMC, is to continue performing his duties as president during a 14-day quarantine. In the event of Trump becoming incapacitated, Vice President Mike Pence could take over the duties of president via the 25th Amendment of the US Constitution. The Pence family all tested negative as of yesterday and there were no changes regarding Pence's campaign events.
'''

text =  "headline: " + text

max_len = 256

encoding = tokenizer.encode_plus(text, return_tensors = "pt")
input_ids = encoding["input_ids"].to(device)
attention_masks = encoding["attention_mask"].to(device)

beam_outputs = model.generate(
    input_ids = input_ids,
    attention_mask = attention_masks,
    max_length = 64,
    num_beams = 3,
    early_stopping = True,
)

result = tokenizer.decode(beam_outputs[0])
print(result)

<pad> What's the Difference Between Microgravity and Zero Gravity?</s>
